# FreeYourHandv2.1.2

---

v2.1：

- 加入了对```HTTP500```问题的处理

v2.1.1:

- 修复了合并后缺失文件的问题

v2.1.2:

- 加入了对偶尔下载弹窗未弹出的处理

- 修复了下一页未成功加载时卡刷新的问题

- 开始时增加了当前时间范围的输出方便报错时确认时间

## 导入库
直接运行就好。

In [22]:
import os
import re
from datetime import datetime
from time import sleep
from bs4 import BeautifulSoup as bs
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

# ！！！设置参数

---

这部分是全部的参数设置，大部分是设置好了的，但仍有几个路径需要你自己调整。

**请写好参数后再运行**

In [2]:
# 点击按钮的睡眠时间,2即可，会在出现按钮后等待2秒，点击，再等待两秒。
button_sleep_time = 2

# 是否是中文网页，如果是，那么输入True,不是则False。
isChinese = False

# 浏览器用的哪一个就留哪一个并去掉前面的#，然后把另外的都删掉！！或者前面加个#注释掉，弹出的浏览器不要关
driver = webdriver.Firefox()

# Initialization....
print('Starting driver......')
driver.get('http://www.99885.net')

# setting up waitting time
wait = WebDriverWait(driver, 10)

Starting driver......


## 定义将要使用的函数

In [3]:
# utility functions

def get_page_soup():
    '''
    get page source and turn into Beautiful soup.
    '''
    return bs(driver.page_source, 'html.parser')


def is_more_pages():
    '''
    test for if there is next page.
    '''
    li = get_page_soup()
    li_class = li.select_one('.pagination > li:nth-last-child(2)').get('class')
    
    return (li_class == None)

def click_button(CSS_Selector):
    '''
    click a button by its CSS_Slector.
    '''
    global button_sleep_time
    wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, CSS_Selector)))
    sleep(button_sleep_time)
    button = driver.find_element(By.CSS_SELECTOR, CSS_Selector)
    button.click()
    # waitting time after click button
    sleep(button_sleep_time)
    
    return None


def download_files():
    '''
    Downloading file after selection.
    '''
    global isChinese
    
    if isChinese:
        request_complete = '请求完成'
    else:
        request_complete = 'Request complete'
        
    not_start_downloading = True
    counter = 0
    failed = False
    
    while not_start_downloading:    
        try:
            # click (...) button
            click_button('.uxf-dot-three')

            # click Text_only button
            click_button('div.row-flex:nth-child(6) > div:nth-child(5) > a:nth-child(1) > span:nth-child(1) > span:nth-child(1)')
            
            if failed:
                sleep(5)
            # click Deselect items when done
            click_button("[id^='deselectCheckBox_']")

            # click continue
            click_button("[id^='submitButton_']")
            
            not_start_downloading = False
        except:
            if counter >= 3:
                print('You may excess your limit, please restart.')
                raise Exception("You may excess you limit......")
            print('Dowdloading cell not appearing, refresh page now.')
            driver.refresh()
            sleep(5)
            failed = True
            counter += 1
    sleep(10)
    
    # siwtch to downloading page
    driver.switch_to.window(driver.window_handles[-1])
    not_finished = (get_page_soup().select_one('.header_border_bottom > h1:nth-child(1)').text) != request_complete
    waiting_time = 0
    
    while not_finished:
        sleep(10)
        waiting_time += 10
        not_finished = (get_page_soup().select_one('.header_border_bottom > h1:nth-child(1)').text) != request_complete
        if waiting_time >= 150:
            print("Waiting exceed 150, \033[1;32mRefresh\033[0m page...")
            driver.refresh()
            waiting_time = 0
            sleep(5)
        
    print('\033[1;32mRequest complete!\033[0m')
    sleep(3)
    # close the downloading tab and switch back.
    driver.close()
    driver.switch_to.window(driver.window_handles[-1])
    
    return None


def get_current_page_num():
    '''
    get current page number.
    '''
    page = get_page_soup()
    
    return int(page.select('#pageNbrField')[0].get('value'))


def input_num(num, CSS_Selector):
    '''
    input a number to the box
    '''
    box = driver.find_element(By.CSS_SELECTOR, CSS_Selector)
    box.clear()
    box.send_keys(num)
    box.send_keys(Keys.RETURN)
    
    return None


def read_html_to_soup(file_path):
    '''
    read a html file into soup.
    '''
    with open(file_path, 'r') as file:
        html_content = file.read()
        
    soup = bs(html_content, 'html.parser')
    
    return soup

## 开始下载

In [21]:
# switch to right-most tab
driver.switch_to.window(driver.window_handles[-1])

# setting up parameters
selected_pages = 0

# downloading....
print('========================================')
print('Scraper Working now......')
print('Time Range : ', get_page_soup().select_one('#dateFilter-div > ul:nth-child(1) > li:nth-child(1) > div:nth-child(2) > span:nth-child(1)').get_text())
print('========================================')
while is_more_pages():
    current_page = get_current_page_num()
    print(f'------Now in page \033[1;32m{current_page}\033[0m------')
    # click "select"
    click_button('#mlcbAll')
    selected_pages += 1
    print(f'Page {current_page} Selected')
    
    # start downloading if selected 5 pages
    if selected_pages == 5:
        print(f'\033[1;32mStart downloading page {current_page - 4}-{current_page}\033[0m')
        download_files()
        selected_pages = 0
    
    # turn to next page
    print('Click next page.')
    click_button('.uxf-right-open-large')
    
    
    # test for if the next page is loaded.
    total_waiting_time = 0
    not_next_page = True
    
    while not_next_page:
        sleep(3)
        total_waiting_time += 3
        
        try:
            if current_page == get_current_page_num() - 1:
                not_next_page = False
        except:
            h1 = get_page_soup().select_one('body > h1').get_text()
            if h1 == 'HTTP Status 500 – Internal Server Error':
                print(f'\033[1;31mHTTP Status 500 Error\033[0m encounterd while trying turn to page {current_page + 1}')
                print('Refresh page...')
                driver.refresh()
                sleep(3)
            
        if (total_waiting_time > 29) and not_next_page:
            print('Waiting more than 30s for next page, \033[1;31mrefresh page\033[0m now.')
            driver.refresh()
            sleep(3)
            wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, '#pageNbrField')))
            input_num(current_page + 1, '#pageNbrField')
            total_waiting_time = 0
            

# downloading last page
print('Selecting last page.')
click_button('#mlcbAll')
selected_pages += 1
print(f'Start downloading page {current_page + 1}-{get_current_page_num()}')
download_files()
print('\n\033[1;32mAll Task Finished!\033[0m')

Scraper Working now......
Time Range :  1931-12-01 - 1931-12-31
------Now in page 36------
Page 36 Selected
Click next page.
------Now in page 37------
Page 37 Selected
Click next page.
------Now in page 38------
Page 38 Selected
Click next page.
------Now in page 39------
Page 39 Selected
Click next page.
------Now in page 40------
Page 40 Selected
Start downloading page 36-40
Request complete!
Click next page.
------Now in page 41------
Page 41 Selected
Click next page.
------Now in page 42------
Page 42 Selected
Click next page.
------Now in page 43------
Page 43 Selected
Click next page.
------Now in page 44------
Page 44 Selected
Click next page.
------Now in page 45------
Page 45 Selected
Start downloading page 41-45
Request complete!
Click next page.
------Now in page 46------
Page 46 Selected
Click next page.
------Now in page 47------
Page 47 Selected
Click next page.
------Now in page 48------
Page 48 Selected
Click next page.
------Now in page 49------
Page 49 Selected
Click

## 结束

In [12]:
# quit the driver.
driver.quit()

# 文件合并与排序

---

该jupyter notebook 用于下载完成后的合并与排序。

代码其实是Chatgpt4帮我写的，不过这部分代码只需要基础的python库所以应该都没问题。

## 设置文件路径

第一步是新建一个文件夹，把所有下载的文件放进去。

然后我推荐你复制进去留个备份而不是剪切进去防止代码有问题全丢了白干。

第二步，记住这个文件夹的绝对路径。比如，你把所有文件放在了D盘的merge文件夹，

然后你就应该在下面的```""```里面填入```D:\\merge\```

我不确定我这路径里斜线写的对不对，更方便的，你先输入```D:```，然后按一下```tab```，应该就会出现D盘的路径了，再按一下，就会出现D盘下的所有文件了，选择merge/就好了。

In [23]:
# 这里是我自己的路径，我放到了nolan用户下的downloads文件下的111文件夹里面。你删掉""里面的就行，可别把“”删了。
# 注意一定是英文输入法
directory = "/home/nolan/Downloads/111/"

接下来直接运行就行了。

## 去除每个文件最后的版权内容

In [24]:
def remove_last_section_if_url_present(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.readlines()

    separator = "____________________________________________________________\n"
    url_text = "http://about.proquest.com/go/pqissupportcontact"
    if separator in content:
        last_separator_index = len(content) - 1 - content[::-1].index(separator)
        last_section = content[last_separator_index:]

        if any(url_text in line for line in last_section):
            content = content[:last_separator_index]

    with open(file_path, 'w', encoding='utf-8') as file:
        file.writelines(content)

def process_folder(folder_path):
    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):
            file_path = os.path.join(folder_path, filename)
            remove_last_section_if_url_present(file_path)

process_folder(directory)
print('\n\033[1;32mAll Task Finished!\033[0m')


All Task Finished!


## 合并

In [25]:
def extract_date(text):
    if isChinese:
        match = re.search(r'出版日期: (\w+ \d{1,2}, \d{4})', text)
    else:
        match = re.search(r'Publication date: (\w+ \d{1,2}, \d{4})', text)
    if match:
        date_str = match.group(1)
        # 将日期字符串转换为 datetime 对象，然后格式化为 YYYYMM
        return datetime.strptime(date_str, '%b %d, %Y').strftime('%Y%m')
    return None

def merge_files(folder_path):
    files_data = {}  # 用于存储年月和对应的文件内容

    for filename in os.listdir(folder_path):
        if filename.endswith('.txt'):
            with open(os.path.join(folder_path, filename), 'r', encoding='utf-8') as file:
                content = file.read()
                date = extract_date(content)

                if date:
                    if date in files_data:
                        files_data[date].append(content)
                    else:
                        files_data[date] = [content]

    for date, contents in files_data.items():
        merged_filename = f"{date}.txt"
        with open(os.path.join(folder_path, merged_filename), 'w', encoding='utf-8') as merged_file:
            merged_file.write('\n____________________________________________________________\n'.join(contents))

            
def delete_specific_files(directory, keyword):
    # Walk through the directory
    for root, dirs, files in os.walk(directory):
        for file in files:
            # Check if the keyword is in the filename
            if keyword in file and file.endswith('.txt'):
                # Construct the full file path
                file_path = os.path.join(root, file)
                # Delete the file
                os.remove(file_path)

merge_files(directory)
delete_specific_files(directory, 'ProQuestDocuments')
print('\n\033[1;32mAll Task Finished!\033[0m')


All Task Finished!


## 排序

---

下面的代码进行排序。

In [26]:
def parse_date(article):
    date_match = re.search(r'Publication date: (\w+ \d+, \d+)', article)
    if date_match:
        date_str = date_match.group(1)
        try:
            return datetime.strptime(date_str, '%b %d, %Y')
        except ValueError:
            return None
    return None

def sort_articles_by_date(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        content = file.read()

    articles = content.split('____________________________________________________________')

    # Filter out articles without a valid date
    articles_with_date = [article for article in articles if parse_date(article) is not None]
    articles_without_date = [article for article in articles if parse_date(article) is None]

    sorted_articles = sorted(articles_with_date, key=parse_date)

    # Append the articles without a date to the end
    sorted_articles.extend(articles_without_date)

    return sorted_articles


for filename in os.listdir(directory):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory, filename)
        sorted_articles = sort_articles_by_date(file_path)
        with open(file_path, 'w', encoding='utf-8') as file:
            file.write('____________________________________________________________'.join(sorted_articles))
            
            
print('\n\033[1;32mAll Task Finished!\033[0m')


All Task Finished!
